In [16]:
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate, cross_val_score
import pandas as pd
import numpy as np
import os
os.chdir('C:\master_data_science')
df = pd.read_csv('titanic.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
X = df[['Pclass', 'SibSp', 'Parch', 'Fare']]
y = df['Survived']

In [18]:
X, y

(     Pclass  SibSp  Parch     Fare
 0         3      1      0   7.2500
 1         1      1      0  71.2833
 2         3      0      0   7.9250
 3         1      1      0  53.1000
 4         3      0      0   8.0500
 ..      ...    ...    ...      ...
 886       2      0      0  13.0000
 887       1      0      0  30.0000
 888       3      1      2  23.4500
 889       1      0      0  30.0000
 890       3      0      0   7.7500
 
 [891 rows x 4 columns],
 0      0
 1      1
 2      1
 3      1
 4      0
       ..
 886    0
 887    1
 888    0
 889    1
 890    0
 Name: Survived, Length: 891, dtype: int64)

In [19]:
algorithms = {
    'knn': KNeighborsClassifier(),
    'lr': LogisticRegression(solver = 'liblinear'),
    'dt': DecisionTreeClassifier()
}

# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
metrics = [
    'accuracy',
    'roc_auc',
    'precision'
]

params = {
    'knn': {
        'n_neighbors': [3, 5, 10],
        'weights': ['uniform', 'distance']
    },
    'dt': {
        'max_depth': [None, 5, 10, 20]
    }
}


class Judge:
    
    def __init__(self):
        return None
    
    def set_algorithms(self, algorithms):
        self.algorithms = algorithms
        return None
    
    def set_data(self, X, y):
        self.X = X
        self.y = y
        return None
    
    def set_metrics(self, metrics):
        self.metrics = metrics
        return None
        
    def set_params(self, params):
        self.params = params
        return None
    
    def get_performance(self, metric, algorithm, grid):
        best_algorithm = algorithm
        if bool(grid):
            best_algorithm = GridSearchCV(estimator = algorithm, param_grid = grid)  # inner loop
        scores = cross_validate(best_algorithm, X = self.X, y = self.y, scoring = metric, cv = 10)  # outer_loop
        score = np.mean(scores['test_score'])
        score = round(score, 2)
        
        return score
        
    def get_table(self):
        metrics_results = {}
        for metric in self.metrics:
            algorithms_results = {}
            for label, algorithm in self.algorithms.items():

                grid = {}
                if label in self.params.keys():
                    grid = self.params[label]
                algorithms_results[label] = self.get_performance(metric, algorithm, grid)
            metrics_results[metric] = algorithms_results
            
        df = pd.DataFrame.from_dict(metrics_results)
                
        return df
    
    
j = Judge()
j.set_algorithms(algorithms)
j.set_data(X, y)
j.set_metrics(metrics)
j.set_params(params)
j.get_table()

,accuracy,roc_auc,precision
knn,0.67,0.69,0.58
lr,0.69,0.71,0.63
dt,0.70,0.71,0.68
